In [1]:
import psycopg2
import pandas as pd
from pathlib import Path
import csv


conn = psycopg2.connect(database="f1_halo",
                       host="localhost",
                       user="postgres",
                       password = "postgres",
                       port="5432")

In [2]:
# Store filepaths into variable
driver_csv = Path("data/drivers.csv")
fcyphases_csv = Path("data/fcyphases.csv")
laps_csv = Path("data/laps.csv")
qualifyings_csv = Path("data/qualifyings.csv")
races_csv = Path("data/races.csv")
retirements_csv = Path("data/retirements.csv")
starterfields_csv = Path("data/starterfields.csv")

In [3]:
tables = ["drivers","fcyphases","laps","qualifyings","races","retirements","starterfields"]
for i in tables:
    cursor = conn.cursor()
    query = f'SELECT * from {i};'
    #print(query)
    cursor.execute(query)
    rows = cursor.fetchall()
    df_name = i + "_df"
    globals()[df_name]= pd.DataFrame(rows)
    print(df_name)
    
    
    #csv for headers
    path = f'data/{i}.csv'
    csv_path = Path(path)
    with open(csv_path, 'r', newline='') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Read the first row
        if len(globals()[df_name].columns) == len(headers):
            globals()[df_name].columns = headers

    #print(df_name)
    print(globals()[df_name].columns)

drivers_df


FileNotFoundError: [Errno 2] No such file or directory: 'data\\drivers.csv'

In [87]:
fcyphases_df.head()

,id,race_id,startracetime,endracetime,startraceprog,endraceprog,startlap,endlap,type
0,1,1,1069,1604,11.107,14.922,12,15,SC
1,2,3,4157,4899,41.010,45.965,42,46,SC
2,3,6,59,332,0.583,2.897,1,3,SC
3,4,6,2157,2710,25.289,29.916,26,30,SC
4,5,7,38,829,0.372,6.864,1,7,SC


In [71]:
#races_df.rename(columns = {0:'race_id',1:'driver_id',2:'team',3:'teamcolor',4:'enginemanufacturer',5:'gridposition',6:'status', 7:'resultposition',8:'completedlaps',9:'speedtrap'}, inplace = True)
#races_df.head()

In [72]:
#starterfields_df.rename(columns = {0:'race_id',1:'driver_id',2:'team',3:'teamcolor',4:'enginemanufacturer',5:'gridposition',6:'status', 7:'resultposition',8:'completedlaps',9:'speedtrap'}, inplace = True)
#starterfields_df.head()

In [89]:
drivers_df.rename(columns = {0:'driver_id'}, inplace = True)
drivers_df.head()

,id,carno,initials,name
0,1,44,HAM,Lewis Hamilton
1,2,3,RIC,Daniel Ricciardo
2,3,6,ROS,Nico Rosberg
3,4,20,MAG,Kevin Magnussen
4,5,14,ALO,Fernando Alonso


In [88]:
newdf = pd.merge(drivers_df, starterfields_df)
newdf

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False